In [2]:
import json
import os
import math
import random
import numpy as np
from tqdm import tqdm

subgraph = 0
B = 125_000_000 # BYTES

out_name = f'traces/trace_{subgraph}'                                       # name of saved trace
config_path = os.getcwd() + f'/topologies/final_topology_{subgraph}/'       # path to switch config files for topology
data_path = 'data/reindexed.json'                                           # path to file containing topology

In [3]:
configs = os.listdir(config_path)

with open(data_path,'r') as f:
    data = json.load(f)

nodes = {}
for node in data['nodeList']:
    nodes[node['id']] = node

sources = [] # access
destinations = [] # kernel or mixed 

for config in configs:
    id = int(config.split('.')[0])

    if nodes[id]['deviceLevel'] == 'Access':
        sources.append(id)
    else:
        destinations.append(id) 

print("num src:",len(sources), "|| num dests:",len(destinations))

num src: 5149 || num dests: 88


In [4]:
FLOW_THROUGHPUT = B                     # BYTES PER SECOND
SIMULATION_TIME = 1000000               # Ns
PAIRS_PER_SRC = {'mu': 1, 'sigma': 0}   # NORMAL DIST
MSG_SIZE = 50_000                       # BYTES
PACKET_SIZE = 1400                      # BYTES
BANDWIDTH = 10_000_000                  # BYTES
PRIO_LEVELS = 3                         # QOS PRIORITIES

s_to_ns = lambda x : int(x * math.pow(10,9))
ns_to_s = lambda x : x * math.pow(10,-9)

In [5]:
parameters = (f"FLOW_THROUGHPUT:{FLOW_THROUGHPUT}__"
          f"SIMULATION_TIME:{SIMULATION_TIME}__"
          f"PAIRS_PER_SRC:{tuple(PAIRS_PER_SRC.values())}__"
          f"MSG_SIZE:{MSG_SIZE}__"
          f"PACKET_SIZE:{PACKET_SIZE}__"
          f"BANDWIDTH:{BANDWIDTH}__"
          f"PRIO_LEVELS:{PRIO_LEVELS}")

print(parameters)

FLOW_THROUGHPUT:1250000000__SIMULATION_TIME:1000000__PAIRS_PER_SRC:(1, 0)__MSG_SIZE:50000__PACKET_SIZE:1400__BANDWIDTH:10000000__PRIO_LEVELS:3


In [6]:
# generate flows
flows = {}
for src in sources:
    num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
    while num_pairs <= 0:
        num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
    
    flows[src] = random.sample(sources, k=num_pairs)

In [13]:
def generate_messages_for_flow(duration_ns):
    total_bytes_for_duration = ns_to_s(duration_ns) * FLOW_THROUGHPUT

    sizes = []

    while sum(sizes) < total_bytes_for_duration:
        sizes.append(int(random.expovariate(1/MSG_SIZE)))
    
    times = np.linspace(0, duration_ns, len(sizes))
    times = [int(x) for x in times]

    return times, sizes

In [8]:
'''
    Generation Logic:
        - For each source - while the current message time is less than the simulation time
            - pick the next message time (exp dist)
            - pick a random dest (uniform)
            - pick a random message (size)
            - split the message into packets based on packets size
            - add packets accumulating transmission time (terminal -> switch)
'''

message_id = 0
messages = []
for src, pairs in tqdm(flows.items()):
     for dst in pairs:
        flow_messages = generate_messages_for_flow(SIMULATION_TIME)

        for size, time in zip(*flow_messages):
            tos = random.randint(0, PRIO_LEVELS-1)

            messages.append({
                'message_id': message_id,
                'src': src,
                'dst': dst,
                'size': size,
                'timestamp':time,
                'tos': tos
            })

        message_id += 1

messages = sorted(messages, key=lambda x:x['timestamp'])

  0%|          | 0/5149 [00:00<?, ?it/s]

100%|██████████| 5149/5149 [00:00<00:00, 9040.52it/s]


In [12]:
unique_packet_id = 0

f =  open(out_name+'_'+parameters, 'w')

for msg in tqdm(messages):
    message_id, src, dst, size, timestamp, tos = msg.values()

    num_packets = math.ceil(size / PACKET_SIZE) # ceil -> padding last packet to always be PACKET_SIZE
    
    packet_time = timestamp
    packets = []
    for _ in range(num_packets):
        packet_info = (
            f"{str(unique_packet_id)} "
            f"{str(message_id)} "
            f"{str(src)} "
            f"{str(dst)} "
            f"{str(PACKET_SIZE)} "
            f"{str(packet_time)} "
            f"{str(tos)}\n"
        )

        f.write(packet_info)

        packet_time += PACKET_SIZE / BANDWIDTH
        unique_packet_id += 1 

f.close()
    

100%|██████████| 133784/133784 [01:58<00:00, 1129.74it/s]
